In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
#from statsmodels.tsa.vector_ar.var_model import VARY
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed

import pyarrow
import concurrent.futures
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from itertools import product

import statsmodels

In [ ]:
datafolder = '/home/hoyinchan/blue/Data/data2022/'
labX4 = pd.read_parquet(datafolder+'allmedlabxx.parquet')

In [ ]:
labX = pd.read_parquet(datafolder+'alllab.parquet')

In [ ]:
labX4['LAB_LOINC'].unique()

In [ ]:
dummy_table = labX4[['ID', 'MEDADMIN_CODE_NEW']].drop_duplicates()

In [ ]:
confound_med_HK =    {'MED:ATC:C03DA': 'eplerenone(HK):C03DA'}
confound_med_LK =    {'MED:ATC:A07DA': 'DiphenoxylateLoperamide(LK):A07DA'}
confound_med_HCa =    {'MED:ATC:M05BA': 'PamidronateZoledronate(HCa):M05BA'}
confound_med_LNa =    {'MED:ATC:C03CA': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
                      'MED:ATC:C03AA': 'hydrochlorothiazide(LNa):C03AA',
                      'MED:ATC:C03BA': 'chlorthalidone(LNa):C03BA'}

In [ ]:
loincs_cofounder = ['LG6033-7', #AST       
                    'LG49755-8'] #uric acid

loincs_dependents = ['LG49864-8', #Ca
                     'LG11363-5', #Na
                     'LG49936-4', 'LG6657-3'] #K, $sCr

In [ ]:
dummy_table['if_confound_med_HK'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_HK.keys())))
dummy_table['if_confound_med_LK'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_LK.keys())))
dummy_table['if_confound_med_HCa'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_HCa.keys())))
dummy_table['if_confound_med_LNa'] =  dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_LNa.keys())))

In [ ]:
dummy_table = dummy_table.drop('MEDADMIN_CODE_NEW',axis=1).groupby('ID').sum().astype(bool).reset_index()

In [ ]:
dummy_table = dummy_table.merge(labX[['ID']].drop_duplicates(), on='ID',how='outer').fillna(False)

In [ ]:
K_bound =  [3.2, 4.4] # 2823-3
Na_Bound = [132, 144] # 2951-2
Ca_Bound = [8, 9.3] # 17861-6

In [ ]:
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG49936-4'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_K'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG11363-5'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_Na'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG49864-8'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_Ca'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG6657-3'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_sCr'},axis=1)

In [ ]:
dummy_table['low_K'] = dummy_table['last_K'] <= K_bound[0]
dummy_table['high_K'] = dummy_table['last_K'] >= K_bound[1]
dummy_table['low_Na'] = dummy_table['last_Na'] <= Na_Bound[0]
dummy_table['high_Na'] = dummy_table['last_Na'] >= Na_Bound[1]
dummy_table['low_Ca'] = dummy_table['last_Ca'] <= Ca_Bound[0]
dummy_table['high_Ca'] = dummy_table['last_Ca'] >= Ca_Bound[1]

In [ ]:
#dummy_table = dummy_table.drop(['last_K','last_Na', 'last_Ca'],axis=1)

In [ ]:
final_table = labX[labX['LAB_LOINC'].str.contains('|'.join(loincs_cofounder+loincs_dependents)).fillna(False)][['ID', 'DAYS_SINCE_ADMIT', 'LAB_LOINC', 'RESULT_NUM']]

In [ ]:
final_table = final_table.groupby(['ID', 'DAYS_SINCE_ADMIT', 'LAB_LOINC']).mean().reset_index()

In [ ]:
loincs_cofounder+loincs_dependents

In [ ]:
final_table['LAB_LOINC'].unique()

In [ ]:
pivot_table = final_table.pivot_table(index=['ID', 'DAYS_SINCE_ADMIT'], 
                                      columns='LAB_LOINC', 
                                      values='RESULT_NUM').reset_index()

In [ ]:
pivot_table = pivot_table.merge(dummy_table, on='ID', how='left')

In [ ]:
pivot_table

In [ ]:
pivot_table['ID2'] = pd.factorize(pivot_table['ID'])[0]

In [ ]:
pivot_table[['if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca']] = pivot_table[['if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca']].astype(int)

In [ ]:
pivot_table

In [ ]:
# loinc_to_label = {
#     '1920-8': 'AST',
#     '3084-1': 'UA',
#     '17861-6': 'Ca',
#     '2951-2': 'Na',
#     '2823-3': 'K',
#     '2160-0': 'sCr'
# }

loinc_to_label = {
    'LG6033-7': 'AST',
    'LG49755-8': 'UA',
    'LG49864-8': 'Ca',
    'LG11363-5': 'Na',
    'LG49936-4': 'K',
    'LG6657-3': 'sCr'
}

pivot_table = pivot_table.rename(columns=loinc_to_label)

In [ ]:
pivot_table.to_parquet(datafolder+'confounder_regression.parquet')

In [ ]:
pivot_table.columns

In [ ]:
pivot_table.columns